<a href="https://colab.research.google.com/github/scon012/MLEExercise/blob/main/DuckDB_Tech_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup the environment

In [6]:
!pip install pyarrow pandas
!pip install duckdb --pre --upgrade
!pip install duckdb-engine
!pip install jupysql

import pyarrow.parquet as pq
import pandas
import glob
import duckdb
import sqlalchemy


Clean up the source files

In [12]:
!rm -r "stats_nz_data"

Download and store the data files
* https://www.stats.govt.nz/assets/Uploads/Business-employment-data/Business-employment-data-March-2024-quarter/Download-data/business-employment-data-march-2024-quarter.zip
* https://www.stats.govt.nz/assets/Uploads/Business-financial-data/Business-financial-data-March-2024-quarter/Download-data/business-financial-data-march-2024.zip

In [19]:
employment_data_file = "https://www.stats.govt.nz/assets/Uploads/Business-employment-data/Business-employment-data-March-2024-quarter/Download-data/business-employment-data-march-2024-quarter.zip"
financial_data_file = "https://www.stats.govt.nz/assets/Uploads/Business-financial-data/Business-financial-data-March-2024-quarter/Download-data/business-financial-data-march-2024.zip"

# Create a folder for the zip files
#       Should have code to check if already exists
!mkdir stats_nz_data

# Download the zips
#       Should have code to check if already exists
!wget -O stats_nz_data/employment_data.zip {employment_data_file}
!wget -O stats_nz_data/financial_data.zip {financial_data_file}

# Extract the zip files
!unzip -o -d "stats_nz_data" "stats_nz_data/employment_data.zip"
!unzip -o -d "stats_nz_data" "stats_nz_data/financial_data.zip"

# Rename the files to simple filenames
!mv 'stats_nz_data/machine-readable-business-employment-data-mar-2024-quarter.csv' 'stats_nz_data/employment.csv'
!mv 'stats_nz_data/business-financial-data-march-2024-csv.csv' 'stats_nz_data/financial.csv'

# Test that the files have been extracted correctly
#     I'm not going to do this

Load DuckDB with csv file

In [39]:
# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# Some DuckDB setup
conn = duckdb.connect()
# %sql conn --alias duckdb

# enable automatic query parallelization
conn.execute("PRAGMA threads=2")

# enable caching of parquet metadata
conn.execute("PRAGMA enable_object_cache")

#con.execute("CREATE TABLE ontime AS SELECT * FROM 'https://support.staffbase.com/hc/en-us/article_attachments/360009197011/username-password-recovery-code.csv';")
conn.execute("CREATE TABLE employment AS SELECT * FROM 'stats_nz_data/employment.csv';")
conn.execute("CREATE TABLE financial AS SELECT * FROM 'stats_nz_data/financial.csv';")

%sql conn --alias duckdb
%sql duckdb://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Switching to connection 'duckdb://'

I would like to see them loading into a table with all character columns, performing checks on the data and then loading to the final table with the correct data types. More work but a lot better at catching errors.

Query the data

In [40]:
%%time
conn.execute("SELECT * FROM employment LIMIT 5").df()

CPU times: user 7.13 ms, sys: 1 ms, total: 8.13 ms
Wall time: 7.38 ms


,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,Subject,Group,Series_title_1,Series_title_2,Series_title_3,Series_title_4,Series_title_5
0,BDCQ.SEA1AA,2011.06,80078.0,None,F,Number,0,Business Data Collection - BDC,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual,None,None
1,BDCQ.SEA1AA,2011.09,78324.0,None,F,Number,0,Business Data Collection - BDC,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual,None,None
2,BDCQ.SEA1AA,2011.12,85850.0,None,F,Number,0,Business Data Collection - BDC,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual,None,None
3,BDCQ.SEA1AA,2012.03,90743.0,None,F,Number,0,Business Data Collection - BDC,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual,None,None
4,BDCQ.SEA1AA,2012.06,81780.0,None,F,Number,0,Business Data Collection - BDC,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual,None,None


In [25]:
%%time
conn.execute("SELECT * FROM financial LIMIT 5").df()

CPU times: user 6.04 ms, sys: 1.01 ms, total: 7.05 ms
Wall time: 6.97 ms


,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,Subject,Group,Series_title_1,Series_title_2,Series_title_3,Series_title_4,Series_title_5
0,BDCQ.SF1AA2CA,2016.06,1116.386,None,F,Dollars,6,Business Data Collection - BDC,Industry by financial variable (NZSIOC Level 2),Sales (operating income),Forestry and Logging,Current prices,Unadjusted,None
1,BDCQ.SF1AA2CA,2016.09,1070.874,None,F,Dollars,6,Business Data Collection - BDC,Industry by financial variable (NZSIOC Level 2),Sales (operating income),Forestry and Logging,Current prices,Unadjusted,None
2,BDCQ.SF1AA2CA,2016.12,1054.408,None,F,Dollars,6,Business Data Collection - BDC,Industry by financial variable (NZSIOC Level 2),Sales (operating income),Forestry and Logging,Current prices,Unadjusted,None
3,BDCQ.SF1AA2CA,2017.03,1010.665,None,F,Dollars,6,Business Data Collection - BDC,Industry by financial variable (NZSIOC Level 2),Sales (operating income),Forestry and Logging,Current prices,Unadjusted,None
4,BDCQ.SF1AA2CA,2017.06,1233.700,None,F,Dollars,6,Business Data Collection - BDC,Industry by financial variable (NZSIOC Level 2),Sales (operating income),Forestry and Logging,Current prices,Unadjusted,None


Describe Data

*   Description of the tables(s)
*   Data Dictionary


In [23]:
conn.execute("DESCRIBE employment;").df()

,column_name,column_type,null,key,default,extra
0,Series_reference,VARCHAR,YES,None,None,None
1,Period,DOUBLE,YES,None,None,None
2,Data_value,DOUBLE,YES,None,None,None
3,Suppressed,VARCHAR,YES,None,None,None
4,STATUS,VARCHAR,YES,None,None,None
5,UNITS,VARCHAR,YES,None,None,None
6,Magnitude,BIGINT,YES,None,None,None
7,Subject,VARCHAR,YES,None,None,None
8,Group,VARCHAR,YES,None,None,None
9,Series_title_1,VARCHAR,YES,None,None,None


In [41]:
conn.execute("DESCRIBE financial;").df()

,column_name,column_type,null,key,default,extra
0,Series_reference,VARCHAR,YES,None,None,None
1,Period,DOUBLE,YES,None,None,None
2,Data_value,DOUBLE,YES,None,None,None
3,Suppressed,VARCHAR,YES,None,None,None
4,STATUS,VARCHAR,YES,None,None,None
5,UNITS,VARCHAR,YES,None,None,None
6,Magnitude,BIGINT,YES,None,None,None
7,Subject,VARCHAR,YES,None,None,None
8,Group,VARCHAR,YES,None,None,None
9,Series_title_1,VARCHAR,YES,None,None,None


Changing to using magic SQL

In [50]:
%%sql
DROP TABLE IF EXISTS employment;
DROP TABLE IF EXISTS financial;

CREATE TABLE employment AS SELECT * FROM 'stats_nz_data/employment.csv';
CREATE TABLE financial AS SELECT * FROM 'stats_nz_data/financial.csv';

Running query in 'duckdb://'

Count


In [55]:
%%sql
SELECT COUNT(*) AS employment_count FROM employment;
SELECT COUNT(*) AS financial_count FROM financial;

Running query in 'duckdb://'

financial_count
7635


### **Question 1**
Of the industries where salaries and wages data did NOT exist in 2016 and only appeared later, which industry had the highest average value for actual filled jobs across time and what was that value?

In [ ]:
%%sql

-- Answer question here